In [1]:
import numpy as np

In [19]:
class Layer(object):
    
    def __init__(self, inputs, output_size, activation="sigmoid"):
        
        self.all_activations = {
            "none": lambda z: z,
            "sigmoid": lambda z: 1/(1 + np.exp(-z)),
            "softmax": lambda z: np.exp(z)/np.exp(z).sum()
        }
        
        self.activation = self.all_activations.get(activation)
        
        self.input_size = inputs.shape[1]
        self.output_size = output_size
        
        self.weights = np.random.uniform(-0.1, 0.1, size=(self.input_size, self.output_size))
        
        self.inputs = inputs
        self.raw_outputs = np.matmul(self.inputs, self.weights)
        self.outputs = self.activation(self.raw_outputs)
    
    def update(self, grads):
        pass
    
    

In [20]:
X = np.random.normal(0, 1, size=(1000, 20))

In [21]:
layer1 = Layer(X, 10, "sigmoid")

In [25]:
layer1.outputs

array([[0.4794893 , 0.52408661, 0.60534428, ..., 0.41116653, 0.54642949,
        0.45923938],
       [0.48979275, 0.42032097, 0.40922738, ..., 0.39689806, 0.47133026,
        0.47805079],
       [0.61821863, 0.52032694, 0.49502766, ..., 0.46977009, 0.41468839,
        0.50719079],
       ...,
       [0.5098662 , 0.47108746, 0.52990785, ..., 0.57320637, 0.47352743,
        0.54921469],
       [0.56148078, 0.49170364, 0.6390888 , ..., 0.41181242, 0.44571565,
        0.43542324],
       [0.48568229, 0.3709448 , 0.38438419, ..., 0.47958778, 0.41786065,
        0.5557361 ]])